<a href="https://colab.research.google.com/github/sebastian-dv/CSE-151A-Project/blob/main/temp_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 151A Group Project
# Names

- Pranav Prabu
- Sebastian Diaz
- Jou-Chih Chang
- Juan Yin
- Irving Zhao
- Xianzhe Guo
- Tiankuo Li

# Abstract
This project looks to investigate and find key factors that exist in determining the different causes and factors that hint towards the existence of a specific disease. This project will be accomplished by using the dataset SUPPORT2 in the UC Irvine Machine Learning Repository, using discriminative factors such as blood type, chemical levels, and more to isolate correlations. This will be done through the 42 features offered by the set, looking to differentiate between nine disease categories: acute respiratory failure, chronic obstructive pulmonary disease, congestive heart failure, liver disease, coma, colon cancer, lung cancer, multiple organ system failure with malignancy, and multiple organ system failure with sepsis. We will use the feature set from the listed dataset to find specific relations between certain features that will indicate towards the existence of critical diseases found today.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Data Wrangling/Cleaning and Exploratory Data Analysis (EDA)

In [2]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo
# fetch dataset
support2 = fetch_ucirepo(id=880)

First we take a look at the descriptions of the variables in this dataset:

In [4]:
pd.set_option('display.max_colwidth', None)

full_info = pd.DataFrame({
    'Variable Name': support2.variables['name'],
    'Description': support2.variables['description']
})
full_info

,Variable Name,Description
0,id,None
1,age,Age of the patients in years
2,death,Death at any time up to National Death Index (NDI) data on 31 of December of 1994. Some patients are discharged before the end of the study and are not followed up. The authors looked up the information about death.
3,sex,"Gender of the patient. Listed values are {male, female}."
4,hospdead,Death in hospital
5,slos,Days from Study Entry to Discharge
6,d.time,Days of follow-up
7,dzgroup,"The patient's disease sub category amogst ARF/MOSF w/Sepsis, CHF, COPD, Cirrhosis, Colon Cancer, Coma, Lung Cancer, MOSF w/Malig."
8,dzclass,"The patient's disease category amongst ""ARF/MOSF"", ""COPD/CHF/Cirrhosis"", ""Cancer"", ""Coma""."
9,num.co,The number of simultaneous diseases (or comorbidities) exhibited by the patient. Values are ordinal with higher values indicating worse condition and chances of survival.


In [5]:
df = pd.read_csv('https://archive.ics.uci.edu/static/public/880/data.csv')

df = df[['age','sex','death','dzgroup','scoma','race','sps','aps','diabetes','dementia','ca','meanbp','wblc','hrt','resp','temp','pafi','alb','bili','crea','sod','ph']]
df

,age,sex,death,dzgroup,scoma,race,sps,aps,diabetes,dementia,...,wblc,hrt,resp,temp,pafi,alb,bili,crea,sod,ph
0,62.84998,male,0,Lung Cancer,0.0,other,33.898438,20.0,0,0,...,6.000000,69.0,22.0,36.00000,388.00000,1.799805,0.199982,1.199951,141.0,7.459961
1,60.33899,female,1,Cirrhosis,44.0,white,52.695312,74.0,0,0,...,17.097656,112.0,34.0,34.59375,98.00000,NaN,NaN,5.500000,132.0,7.250000
2,52.74698,female,1,Cirrhosis,0.0,white,20.500000,45.0,0,0,...,8.500000,88.0,28.0,37.39844,231.65625,NaN,2.199707,2.000000,134.0,7.459961
3,42.38498,female,1,Lung Cancer,0.0,white,20.097656,19.0,0,0,...,9.099609,88.0,32.0,35.00000,NaN,NaN,NaN,0.799927,139.0,NaN
4,79.88495,female,0,ARF/MOSF w/Sepsis,26.0,white,23.500000,30.0,0,0,...,13.500000,112.0,20.0,37.89844,173.31250,NaN,NaN,0.799927,143.0,7.509766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100,66.07300,male,0,ARF/MOSF w/Sepsis,0.0,white,16.296875,22.0,0,0,...,7.399414,104.0,22.0,35.69531,280.00000,3.699707,0.399963,1.099854,131.0,7.459961
9101,55.15399,female,0,Coma,41.0,white,25.796875,31.0,0,0,...,NaN,0.0,8.0,38.59375,218.50000,NaN,NaN,5.899414,135.0,7.289062
9102,70.38196,male,0,ARF/MOSF w/Sepsis,0.0,white,22.699219,39.0,0,0,...,8.398438,83.0,24.0,36.69531,180.00000,NaN,0.399963,2.699707,139.0,7.379883
9103,47.01999,male,1,MOSF w/Malig,0.0,white,40.195312,51.0,0,0,...,7.599609,110.0,24.0,36.39844,428.56250,1.199951,0.399963,3.500000,135.0,7.469727


In [6]:
df.shape

(9105, 22)

In [7]:
print("Number of observations:", len(df))

Number of observations: 9105


In [8]:
df.isnull().sum()

age            0
sex            0
death          0
dzgroup        0
scoma          1
race          42
sps            1
aps            1
diabetes       0
dementia       0
ca             0
meanbp         1
wblc         212
hrt            1
resp           1
temp           1
pafi        2325
alb         3372
bili        2601
crea          67
sod            1
ph          2284
dtype: int64

In [9]:
print(df.dzgroup.unique())
print(df.race.unique())
print(df.ca.unique())

['Lung Cancer' 'Cirrhosis' 'ARF/MOSF w/Sepsis' 'Coma' 'CHF' 'Colon Cancer'
 'COPD' 'MOSF w/Malig']
['other' 'white' 'black' 'hispanic' 'asian' nan]
['metastatic' 'no' 'yes']


In [10]:
df['sex'].replace('female', 0, inplace=True)
df['sex'].replace('male', 1, inplace=True)

In [11]:
ohe = OneHotEncoder()
list1 = ['dzgroup', 'race', 'ca']
for i in list1:
  myohedzgroup = ohe.fit_transform(df[i].values.reshape(-1,1)).toarray()
  myohedzgroup=pd.DataFrame(myohedzgroup, columns=ohe.categories_[0])
  df=df.drop([i], axis=1)
  df=pd.concat([df,myohedzgroup],axis=1)

df = df.dropna(axis = 0, how = 'any')
df.drop(df.columns[32], axis=1)

,age,sex,death,scoma,sps,aps,diabetes,dementia,meanbp,wblc,...,Lung Cancer,MOSF w/Malig,asian,black,hispanic,other,white,metastatic,no,yes
0,62.84998,1,0,0.0,33.898438,20.0,0,0,97.0,6.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6,62.37097,1,1,0.0,17.296875,46.0,0,0,78.0,11.699219,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
17,63.66299,0,1,26.0,40.296875,58.0,0,0,69.0,30.097656,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
22,49.61298,0,0,0.0,31.699219,42.0,0,0,67.0,11.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
25,53.84000,1,1,0.0,14.199219,16.0,0,0,134.0,7.799805,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9091,79.17596,1,0,9.0,22.097656,28.0,0,0,80.0,15.798828,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9099,68.61597,0,0,0.0,15.298828,19.0,0,0,71.0,12.599609,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9100,66.07300,1,0,0.0,16.296875,22.0,0,0,109.0,7.399414,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9103,47.01999,1,1,0.0,40.195312,51.0,0,0,99.0,7.599609,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [16]:
# We first choose all other features excluding the binary variables, and death
features = df.iloc[:,:19].drop(['diabetes','dementia','death'], axis=1)
scaler = MinMaxScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
features

,age,sex,scoma,sps,aps,meanbp,wblc,hrt,resp,temp,pafi,alb,bili,crea,sod,ph
0,0.544507,1.0,0.00,0.333700,0.139860,0.538889,0.046591,0.230000,0.293333,0.435004,0.428063,0.048945,0.001590,0.060112,0.476923,0.675213
1,0.538672,1.0,0.00,0.164275,0.321678,0.433333,0.090846,0.400000,0.373333,0.587086,0.338694,0.153840,0.004769,0.081967,0.338462,0.699633
2,0.554409,0.0,0.26,0.398999,0.405594,0.383333,0.233711,0.360000,0.293333,0.510620,0.163405,0.087410,0.220986,0.153014,0.307692,0.661783
3,0.383276,0.0,0.00,0.311257,0.293706,0.372222,0.085416,0.400000,0.640000,0.750213,0.214032,0.055945,0.063587,0.027324,0.307692,0.599511
4,0.434763,1.0,0.00,0.132662,0.111888,0.744444,0.060566,0.353333,0.293333,0.641462,0.332574,0.118875,0.006359,0.038251,0.476923,0.575091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,0.743362,1.0,0.09,0.213269,0.195804,0.444444,0.122680,0.356667,0.426667,0.401869,0.359755,0.090910,0.003180,0.032788,0.384615,0.661783
3836,0.614738,0.0,0.00,0.143884,0.132867,0.394444,0.097837,0.366667,0.133333,0.456245,0.140031,0.129358,0.007949,0.038251,0.276923,0.711844
3837,0.583764,1.0,0.00,0.154070,0.153846,0.605556,0.057457,0.346667,0.293333,0.401869,0.305109,0.115376,0.004769,0.054642,0.323077,0.675213
3838,0.351693,1.0,0.00,0.397962,0.356643,0.550000,0.059012,0.366667,0.320000,0.478335,0.474242,0.027972,0.004769,0.185809,0.384615,0.687424


In [12]:
pip install scikeras

In [13]:
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_score

In [17]:
# All diseases created by one-hot encoding
targets=df.iloc[:, 19:27]
targets

,ARF/MOSF w/Sepsis,CHF,COPD,Cirrhosis,Colon Cancer,Coma,Lung Cancer,MOSF w/Malig
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
9091,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9099,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9100,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
features.shape

(3840, 16)

In [20]:
y= df.iloc[:, 19:27]
X= features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [21]:
def buildmodel():
    model = Sequential([
        Dense(units = 12, activation = 'sigmoid', input_dim = 16),
        Dense(units = 12, activation = 'sigmoid'),
        Dense(units = 12, activation = 'sigmoid'),
        Dense(units = 8, activation = 'sigmoid')
    ])
    model.compile(optimizer ='SGD', loss='mse')
    return(model)

In [22]:
classifier = buildmodel()
history = classifier.fit(X_train.astype(float), y_train, batch_size = 20, epochs = 100, verbose = 1)
yhat_test = classifier.predict(X_test.astype(float))
yhat_test

Epoch 1/100
154/154 [==============================] - 1s 2ms/step - loss: 0.2395
Epoch 2/100
154/154 [==============================] - 0s 2ms/step - loss: 0.2110
Epoch 3/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1883
Epoch 4/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1705
Epoch 5/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1566
Epoch 6/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1457
Epoch 7/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1372
Epoch 8/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1304
Epoch 9/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1250
Epoch 10/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1206
Epoch 11/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1170
Epoch 12/100
154/154 [==============================] - 0s 2ms/step - loss: 0.1141
Epoch 13/100


array([[0.44976375, 0.12291036, 0.12019792, ..., 0.08369668, 0.08299611,
        0.0963697 ],
       [0.44842595, 0.12429133, 0.12245052, ..., 0.08453457, 0.08256347,
        0.09814504],
       [0.4497598 , 0.12360828, 0.12110924, ..., 0.08408146, 0.08255886,
        0.09703279],
       ...,
       [0.45004228, 0.1232185 , 0.12080356, ..., 0.08366079, 0.08290558,
        0.09662522],
       [0.44901046, 0.12449029, 0.12243301, ..., 0.08430552, 0.08261156,
        0.09812801],
       [0.4491452 , 0.12422108, 0.12213399, ..., 0.08436553, 0.08262543,
        0.097844  ]], dtype=float32)

In [23]:
y_label = y_test.to_numpy()
actual = np.argmax(y_label, axis=1)
pred = np.argmax(yhat_test, axis=1)
print(classification_report(actual,pred))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62       344
           1       0.00      0.00      0.00       103
           2       0.00      0.00      0.00        84
           3       0.00      0.00      0.00        41
           4       0.00      0.00      0.00        16
           5       0.00      0.00      0.00        65
           6       0.00      0.00      0.00        46
           7       0.00      0.00      0.00        69

    accuracy                           0.45       768
   macro avg       0.06      0.12      0.08       768
weighted avg       0.20      0.45      0.28       768



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
